In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True)

df.show(10)s

In [ ]:
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True)

In [ ]:
%pyspark
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.types import DateType

schema = [StructField("marketplace", StringType(), True),\
          StructField("customer_id", IntegerType(), True),\
          StructField("review_id", StringType(), True),\
          StructField("product_id", StringType(), True),\
          StructField("product_parent", IntegerType(), True),\
          StructField("product_title", StringType(), True),\
          StructField("product_category", StringType(), True),\
          StructField("star_rating", IntegerType(), True),\
          StructField("helpful_votes", IntegerType(), True),\
          StructField("total_votes", IntegerType(), True),\
          StructField("vine", StringType(), True),\
          StructField("verified_purchase", StringType(), True),\
          StructField("review_headline", StringType(), True),\
          StructField("review_body", StringType(), True),\
          StructField("review_date", DateType(), True)\
          ,]

schema

In [ ]:
data = StructType(fields=schema)
data

In [ ]:
dataframe = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", schema=data, header=True)
dataframe.show(5)

In [ ]:
review_id_table = dataframe.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])

review_id_table.show(10)

In [ ]:
products = dataframe.select(["product_id", "product_title"])

products.show(10)

In [ ]:
def dfZipWithIndex(df, offset=1, colName="rowId"):


    new_schema = StructType(

        df.schema.fields 
        +[StructField(colName, LongType(), True)]  
    )

    zipped_rdd = df.rdd.zipWithIndex()

    new_rdd = zipped_rdd.map(lambda (row, rowId): (list(row) + [rowId + offset] ))

    return spark.createDataFrame(new_rdd, new_schema)

In [ ]:
from pyspark.sql.types import LongType
customer_table = dfZipWithIndex(customer_table, offset=1, colName="customer_count")

customer_table.show(10)

In [ ]:
reviews_df = dataframe.select(["customer_id", "product_id", "product_title", "star_rating"])

reviews_df.show(10)

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://bigdataassignment.cfllboxbzvib.us-east-2.rds.amazonaws.com:5432/bigdataassignment2"
config = {"user":"root", 
          "password": "passwordroot", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='amazon_grocery_review', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='amazon_grocery_product_info', mode=mode, properties=config)

In [ ]:
customer_table.write.jdbc(url=jdbc_url, table='amazon_grocery_customer_info', mode=mode, properties=config)